<div class="hide">\pagebreak</div>
## Finding the Least Squares Regression Line

In [1]:
# Run this cell to set up the notebook, but please don't change it.

# These lines import the Numpy and Datascience modules.
import numpy as np
from datascience import *

# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)
from matplotlib import patches
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

# These lines load the tests.
from client.api.assignment import load_assignment 
tests = load_assignment('least_squares.ok')

In this exercise, you'll work with a small, abstract dataset of 5 x/y pairs.  You'll see 3 ways to find the least-squares regression line.

Run the next cell to generate the dataset `d` and see a scatter plot.

In [2]:
d = Table().with_columns(
    'x', make_array(0,  1,  2,  3,  4),
    'y', make_array(1, .5, -1,  2, -3))
d.scatter('x')

<div class="hide">\pagebreak</div>
When you run it, the next cell generates sliders that control the slope and intercept of a potential line.  When you adjust a slider, the line will move.

#### Question 1
By moving the line around, make your best guess at the least-squares regression line.  (It's okay if your line isn't exactly right, as long as it's reasonable.)

**Note:** Python will probably take about a second to redraw the plot each time you adjust the slider.  We suggest clicking the place on the slider you want to try and waiting for the plot to be drawn; dragging the slider handle around will cause a long lag.

In [3]:
def plot_line(slope, intercept):
    plt.figure(figsize=(5,5))
    
    endpoints = make_array(-2, 7)
    p = plt.plot(endpoints, slope*endpoints + intercept, color='orange', label='Proposed line')
    
    plt.scatter(d.column('x'), d.column('y'), color='blue', label='Points')
    
    plt.xlim(-4, 8)
    plt.ylim(-6, 6)
    plt.gca().set_aspect('equal', adjustable='box')
    
    plt.legend(bbox_to_anchor=(1.8, .8))

interact(plot_line, slope=widgets.FloatSlider(min=-4, max=4, step=.1), intercept=widgets.FloatSlider(min=-4, max=4, step=.1));

<div class="hide">\pagebreak</div>
You can probably find a reasonable-looking line by just eyeballing it.  But remember: the least-squares regression line minimizes the sum of the squared error made by the line for each point.  Your eye might not be able to judge squared errors very well.

#### Question 2
The next cell produces a more useful plot.  Use it to find a line that's closer to the least-squares regression line.

In [4]:
def plot_line_and_errors(slope, intercept):
    plt.figure(figsize=(5,5))
    points = make_array(-2, 7)
    p = plt.plot(points, slope*points + intercept, color='orange', label='Proposed line')
    ax = p[0].axes
    
    predicted_ys = slope*d.column('x') + intercept
    diffs = predicted_ys - d.column('y')
    for i in np.arange(d.num_rows):
        x = d.column('x').item(i)
        y = d.column('y').item(i)
        diff = diffs.item(i)
        
        if diff > 0:
            bottom_left_x = x
            bottom_left_y = y
        else:
            bottom_left_x = x + diff
            bottom_left_y = y + diff
        
        ax.add_patch(patches.Rectangle(make_array(bottom_left_x, bottom_left_y), abs(diff), abs(diff), color='red', alpha=.3, label=('Squared error' if i == 0 else None)))
        plt.plot(make_array(x, x), make_array(y, y + diff), color='red', alpha=.6, label=('Error' if i == 0 else None))
    
    plt.scatter(d.column('x'), d.column('y'), color='blue', label='Points')
    
    plt.xlim(-4, 8)
    plt.ylim(-6, 6)
    plt.gca().set_aspect('equal', adjustable='box')
    
    plt.legend(bbox_to_anchor=(1.8, .8))

interact(plot_line_and_errors, slope=widgets.FloatSlider(min=-4, max=4, step=.1), intercept=widgets.FloatSlider(min=-4, max=4, step=.1));

<div class="hide">\pagebreak</div>
#### Question 3
Describe the visual criterion you used to find a line in question 2.  (For example, a possible (but incorrect) answer is, "I tried to make the red line for the bottom-right point as small as possible.")

*Write your answer here, replacing this text.*

<div class="hide">\pagebreak</div>
#### Question 4
Does the point at (3, 2) have more or less influence than any other point on the location of the line?

*Write your answer here, replacing this text.*

Now, let's have Python find this line for us.  When we use `minimize`, Python goes through a process similar to the one you might have used in question 2.

But Python can't look at a plot that displays errors!  Instead, we tell it how to find the total squared error for a line with a given slope and intercept.

**Note:** We've been working with the *total* squared error in this exercise because it's easier to visualize.  It is more common to work with the *mean* squared error (or the square root of the mean squared error), as the textbook does.

But it turns out that it doesn't matter whether you minimize the total squared error or the mean squared error.  You'll get the same best line in either case.

That's because the total squared error is just the mean squared error multipled by `d.num_rows`.  So if one line gets a better total squared error than another line, then it also gets a better mean squared error.  In particular, the line with the smallest total squared error is also better than every other line in terms of mean squared error.  That makes it the least squares line.

<div class="hide">\pagebreak</div>
#### Question 5
Define a function called `total_squared_error`.  It should take two numbers as arguments:

1. the slope of some potential line
2. the intercept of some potential line

It should return the total squared error when we use that line to make predictions for the dataset `d`.

In [5]:
def total_squared_error(slope, intercept):
    # Hint: The staff answer computed an array called predictions
    # and an array called errors first.
    predictions = ...
    errors = ...
    ...

In [6]:
_ = tests.grade('q5')

<div class="hide">\pagebreak</div>
#### Question 6
What is the total squared error for the line you found by "eyeballing" the errors in the first question?  What about the question after that, where you had a better visual aid?  (It's okay if the error went up!)

In [8]:
eyeballed_error = ...
aided_error = ...
print("Eyeballed error:", eyeballed_error, "\nAided error:", aided_error)

<div class="hide">\pagebreak</div>
#### Question 7
Use `minimize` to find the actual slope and intercept of the least-squares regression line.

**Note:** `minimize` will return a single array containing the slope as the first element and intercept as the second.

In [9]:
# The staff solution used 1 line of code above here.
slope_from_minimize = ...
intercept_from_minimize = ...
print("Least-squares regression line: predicted_y = {:f}*x + {:f}".format(slope_from_minimize, intercept_from_minimize))

<div class="hide">\pagebreak</div>
#### Question 8
What was the total squared error for that line?

In [13]:
best_total_squared_error = ...
best_total_squared_error

Run the following cell to plot this line and its errors:

In [14]:
plot_line_and_errors(slope_from_minimize, intercept_from_minimize)

<div class="hide">\pagebreak</div>
#### Question 9
Compute the correlation between the `"x"` and `"y"` columns, then use the formula in [13.2](https://www.inferentialthinking.com/chapters/13/2/regression-line.html) to find the slope and intercept of the least-squares regression line.

In [15]:
# The staff solution used 4 lines of code before this.
d_r = ...
slope_from_r = ...
intercept_from_r = ...
print("Regression line computed from the correlation: predicted_y = {:f}*x + {:f}".format(slope_from_r, intercept_from_r))

Compare this with your answer to question 7 to verify that they're both correct.  They will be a little bit different, because `minimize` is by default accurate only to within $0.01$.  (If they're not roughly the same, try computing the total squared error for both; the one with the smaller error is more likely correct!)